## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

df = pd.read_csv('../data/cookie_cats.csv')
df.groupby("version")["retention_7"].mean()

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Нульова гіпотеза (H₀): Переміщення воріт з рівня 30 на рівень 40 не впливає на утримання через 7 днів. Тобто середні показники `retention_7` для обох груп однакові.

Альтернативна гіпотеза (H₁): Переміщення воріт з рівня 30 на рівень 40 впливає на утримання через 7 днів. Тобто середні показники `retention_7` для обох груп різні.

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [2]:
# Розрахунок кількості успіхів (кількість користувачів, що повернулися) та загального розміру вибірок
success_control = df[df["version"] == "gate_30"]["retention_7"].sum()
success_treatment = df[df["version"] == "gate_40"]["retention_7"].sum()
n_control = df[df["version"] == "gate_30"].shape[0]
n_treatment = df[df["version"] == "gate_40"].shape[0]

# Пропорції успіхів
prop_control = success_control / n_control
prop_treatment = success_treatment / n_treatment

# Z-тест
z_stat, p_value = proportions_ztest([success_control, success_treatment], [n_control, n_treatment])

# Довірчі інтервали
ci_control = proportion_confint(success_control, n_control, alpha=0.05, method="normal")
ci_treatment = proportion_confint(success_treatment, n_treatment, alpha=0.05, method="normal")

# Виведення результатів
print(f"z statistic: {z_stat:.3f} \np-value: {p_value:.4f}")
print(f"Довірчий інтервал 95% для групи control: [{ci_control[0]:.3f}, {ci_control[1]:.3f}]")
print(f"Довірчий інтервал 95% для групи treatment: [{ci_treatment[0]:.3f}, {ci_treatment[1]:.3f}]")

z statistic: 3.164 
p-value: 0.0016
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.178, 0.186]


Ми можемо впевнено відхилити нульову гіпотезу (H₀), оскільки p-value набагато менше рівня значущості 𝛼. Це означає, що у нас є статистично значуща різниця між поведінкою користувачів у різних версіях гри. Утримання користувачів через 7 днів для групи `gate_30` та `gate_40` дійсно різниться.


3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [3]:
from scipy.stats import chi2_contingency

# Таблиця 2х2
contingency_table = pd.crosstab(df['version'], df['retention_7'])
print(contingency_table)

retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279


In [4]:
# Виконуємо хі-квадрат тест
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Виводимо результати
print(f"Хі-квадрат: {chi2:.4f} \np-значення: {p:.4f} \nСтупені свободи: {dof}")
print(f"Очікувані значення:\n{expected}")

# Виводимо висновки
if p_value < 0.05:
    print("\nВідхиляємо нульову гіпотезу (Н0): є залежність між версією гри і значенням retention_7..")
else:
    print("\nНульова гіпотеза (H0) не може бути відхилена: значення retention_7 не залежить від версії гри.")

Хі-квадрат: 9.9591 
p-значення: 0.0016 
Ступені свободи: 1
Очікувані значення:
[[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]

Відхиляємо нульову гіпотезу (Н0): є залежність між версією гри і значенням retention_7..


Отримані результати свідчать про те, що переміщення "воріт" з рівня 30 на рівень 40 має статистично значущий вплив на утримання гравців на 7-й день після встановлення гри.

Це важливий результат, адже він показує, що зміна дизайну гри може вплинути на поведінку користувачів. Але також варто врахувати практичну значущість ефекту, щоб зрозуміти, чи зміна справді варта впровадження.